In [1]:
import pandas as pd

mbti= pd.read_csv('../gonzaloalvarez/final_project/mbti_changed.csv')

In [2]:
mbti.head()

,type,posts,type_index,cleaned_text
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,8,enfp and intj moments sportscenter not t...
1,ENTP,'I'm finding the lack of me in these posts ver...,3,im finding the lack of me in these pos...
2,INTP,'Good one _____ https://www.youtube.com/wat...,11,good one of course to which i say i ...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",10,dear intp i enjoyed our conversation the...
4,ENTJ,'You're fired.|||That's another silly misconce...,2,youre fired thats another silly misconcep...


In [3]:
seq_len= 50

In [4]:
from transformers import AutoTokenizer

In [5]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

In [6]:
import numpy as np

In [7]:
Xids= np.zeros((len(mbti), seq_len))
Xmask=np.zeros((len(mbti), seq_len))

In [8]:
for i, sequence in enumerate(mbti['cleaned_text']):
    tokens=tokenizer.encode_plus(sequence, max_length=seq_len, 
                                truncation=True, 
                                padding='max_length')
    Xids[i,:], Xmask[i,:]= tokens['input_ids'], tokens['attention_mask']

In [9]:
arr= mbti.type_index.values
labels=np.zeros((arr.size, arr.max()+1))

In [10]:
labels[np.arange(arr.size), arr]=1

In [11]:
import tensorflow as tf

In [12]:
dataset= tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

In [13]:
for i in dataset.take(1):
    print(i)

(<tf.Tensor: shape=(50,), dtype=float64, numpy=
array([  101.,  4372., 22540.,  1998., 20014.,  3501.,  5312.,  2998.,
       13013.,  2121.,  2025.,  2327.,  2702.,  3248., 26418.,  2015.,
        2054.,  2038.,  2042.,  1996.,  2087.,  2166., 22305.,  2075.,
        3325.,  1999.,  2115.,  2166.,  2006.,  9377.,  2005.,  2087.,
        1997.,  2651.,  2089.,  1996.,  2566.,  2278.,  3325., 10047.,
       16862.,  2063.,  2017.,  1996.,  2197.,  2518.,  2026.,  1999.,
        2546.,   102.])>, <tf.Tensor: shape=(50,), dtype=float64, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])>, <tf.Tensor: shape=(16,), dtype=float64, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])>)


In [14]:
def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

In [15]:
dataset=dataset.map(map_func)

In [16]:
for i in dataset.take(1):
    print(i)

({'input_ids': <tf.Tensor: shape=(50,), dtype=float64, numpy=
array([  101.,  4372., 22540.,  1998., 20014.,  3501.,  5312.,  2998.,
       13013.,  2121.,  2025.,  2327.,  2702.,  3248., 26418.,  2015.,
        2054.,  2038.,  2042.,  1996.,  2087.,  2166., 22305.,  2075.,
        3325.,  1999.,  2115.,  2166.,  2006.,  9377.,  2005.,  2087.,
        1997.,  2651.,  2089.,  1996.,  2566.,  2278.,  3325., 10047.,
       16862.,  2063.,  2017.,  1996.,  2197.,  2518.,  2026.,  1999.,
        2546.,   102.])>, 'attention_mask': <tf.Tensor: shape=(50,), dtype=float64, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])>}, <tf.Tensor: shape=(16,), dtype=float64, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])>)


In [17]:
dataset= dataset.shuffle(100000).batch(32)

In [18]:
ds_len= len(list(dataset))

In [19]:
split= 0.9

train= dataset.take(round(ds_len*split))
val=dataset.skip(round(ds_len*split))

In [20]:
from transformers import TFAutoModel

In [21]:
bert= TFAutoModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [22]:
input_ids= tf.keras.layers.Input(shape=(seq_len,), name='input_ids', dtype='int32')
mask=  tf.keras.layers.Input(shape=(seq_len,), name='attention_mask', dtype='int32')

embeddings= bert(input_ids, attention_mask= mask)[0]

X= tf.keras.layers.GlobalMaxPool1D()(embeddings)
X=tf.keras.layers.BatchNormalization()(X)
X=tf.keras.layers.Dense(128, activation='relu')(X)
X=tf.keras.layers.Dropout(0.1)(X)
X=tf.keras.layers.Dense(32, activation='relu')(X)
y= tf.keras.layers.Dense(16, activation='softmax', name= 'output')(X)

model= tf.keras.Model(inputs=[input_ids, mask], outputs= y)

model.layers[2].trainable= False

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 50)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 50)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
global_max_pooling1d (GlobalMax (None, 768)          0           tf_bert_model[0][0]          

In [24]:
optimizer= tf.keras.optimizers.Adam(0.001)
loss= tf.keras.losses.CategoricalCrossentropy()
acc= tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss= loss, metrics= [acc])


In [25]:
history= model.fit(train,
                  validation_data= val,
                  epochs=3)

Epoch 1/3
245/245 [==============================] - 445s 2s/step - loss: 2.6605 - accuracy: 0.1462 - val_loss: 2.2376 - val_accuracy: 0.2515
Epoch 2/3
245/245 [==============================] - 459s 2s/step - loss: 2.2319 - accuracy: 0.2449 - val_loss: 2.1072 - val_accuracy: 0.2862
Epoch 3/3
245/245 [==============================] - 489s 2s/step - loss: 2.1914 - accuracy: 0.2606 - val_loss: 2.0433 - val_accuracy: 0.3222


In [28]:
model.evaluate(val, verbose=1)

27/27 [==============================] - 37s 1s/step - loss: 2.0240 - accuracy: 0.3257


[2.0240261554718018, 0.3257485032081604]